In [2]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen, urlretrieve
from selenium import webdriver
from tqdm import tqdm
import os, re, time, random
from multiprocessing.dummy import Pool as ThreadPool
import multiprocessing
import pandas as pd
from functools import reduce
import inspect
from selenium.webdriver.common.keys import Keys

# Analyze Pandas usage in Kaggle Kernels

Our goal was to try to construct a coverage metric for Pandas on Ray, in other words, how much of the actual use cases of Pandas do we provide support for. In order to this, we took the following approach

1. Fetch the top 1800 Python kernels from Kaggle sorted by number of votes
2. Convert notebooks to scripts so all kernels are in .py format
3. Use basic regex matching against known Pandas (pd.\*) and Dataframe (df.\*) methods
4. Aggregate counts for most used methods for the full sample space
5. Calculate kernel-level coverage by checking which scripts have full method implementation in our library

## 1. Get urls of top kernels by vote

In [ ]:
browser = webdriver.Chrome(os.getcwd() + '/dependencies/chromedriver')

browser.get("https://www.kaggle.com/kernels?sortBy=votes&group=everyone&pageSize=20&language=Python")
time.sleep(1)

elem = browser.find_element_by_tag_name("body")

no_of_pagedowns = 150

while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    no_of_pagedowns-=1

soup = BeautifulSoup(browser.page_source, "html.parser")

In [ ]:
kernel_links = ['https://www.kaggle.com' + a['href'] + '/code' for a in soup.find_all("a", class_="block-link__anchor") if ('teaching-notebook-for-total-imaging-newbies' not in a['href'] and 'mentions-of-kaggle-on-hacker-news' not in a['href'])]

In [ ]:
len(kernel_links)

## 2. Get url of code files from kernels page

In [ ]:
browser = webdriver.Chrome('/Users/adithya/Documents/university/research/scrapingKaggle/chromedriver')

In [ ]:
def get_code_from_kernel(kernel_link):
    if ('stkbailey' in kernel_link or 
        'mentions-of-kaggle-on-hacker-news' in kernel_link or 
        'keras-rcnn-based-overview-wip' in kernel_link or 
        'why-employees-leave-and-how-to-predict-it' in kernel_link or 
        'boosting-stacking-and-bayes-searching' in kernel_link or
        'resnet50-features-xgboost' in kernel_link or 
        'github-commit-messages' in kernel_link):
        return None
    try:
        browser.get(kernel_link)
        innerHTML = browser.execute_script("return document.body.innerHTML")
        soup = BeautifulSoup(innerHTML, 'html.parser')

        link = soup.find_all("a", class_="script-code-pane__download")[0]['href']

        return link
    except Exception as e:
        print("E", e)
        return None

In [ ]:
def pull_partition(start, end, partition):
    code_links_write_part = []
    for kl in tqdm(kernel_links[start:end]):
        code_links_write_part.append(get_code_from_kernel(kl))
        time.sleep(1)

    code_file_write_part = open('links/code_links_part' + str(partition) + '.txt', 'w')
    for code_link in code_links_write_part:
        code_file_write_part.write("%s\n" % code_link)
        
    print("Finished partition %d" % partition)

In [ ]:
STEP = 100
partitions = [(STEP*i, STEP*(i+1), i) for i in range(len(kernel_links)//STEP + 1)]
for p in partitions[17:]:
    pull_partition(p[0], p[1], p[2])

In [86]:
code_links_read = []
for i in range(19):
    with open('links/code_links_part' + str(i) + '.txt', 'r') as code_file_read:
        code_links_read += code_file_read.read().splitlines()

In [87]:
code_links = list(set(['https://www.kaggle.com' + cl for cl in code_links_read]))
code_links[:5]

['https://www.kaggle.com/kernels/scriptcontent/2205465/download',
 'https://www.kaggle.com/kernels/scriptcontent/1723057/download',
 'https://www.kaggle.com/kernels/scriptcontent/610626/download',
 'https://www.kaggle.com/kernels/scriptcontent/2041567/download',
 'https://www.kaggle.com/kernels/scriptcontent/1184830/download']

## 3. Pull raw code file to local dir

In [88]:
for code_link in tqdm(code_links):
    !wget --content-disposition $code_link -P data

  0%|          | 0/1845 [00:00<?, ?it/s]

--2018-04-24 13:37:42--  https://www.kaggle.com/kernels/scriptcontent/2205465/download
Resolving www.kaggle.com (www.kaggle.com)... 168.62.224.124
Connecting to www.kaggle.com (www.kaggle.com)|168.62.224.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11183 (11K) [application/octet-stream]
Saving to: ‘data/script.ipynb’

script.ipynb        100%[===================>]  10.92K  --.-KB/s    in 0.001s  

2018-04-24 13:37:43 (10.5 MB/s) - ‘data/script.ipynb’ saved [11183/11183]



  0%|          | 1/1845 [00:00<20:09,  1.52it/s]

--2018-04-24 13:37:43--  https://www.kaggle.com/kernels/scriptcontent/1723057/download
Resolving www.kaggle.com (www.kaggle.com)... 168.62.224.124
Connecting to www.kaggle.com (www.kaggle.com)|168.62.224.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5235 (5.1K) [application/octet-stream]
Saving to: ‘data/script.ipynb.1’

script.ipynb.1      100%[===================>]   5.11K  --.-KB/s    in 0s      

2018-04-24 13:37:43 (41.6 MB/s) - ‘data/script.ipynb.1’ saved [5235/5235]



  0%|          | 2/1845 [00:01<17:01,  1.80it/s]

...

100%|█████████▉| 1843/1845 [17:30<00:01,  1.75it/s]

--2018-04-24 13:55:13--  https://www.kaggle.com/kernels/scriptcontent/516760/download
Resolving www.kaggle.com (www.kaggle.com)... 168.62.224.124
Connecting to www.kaggle.com (www.kaggle.com)|168.62.224.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18524 (18K) [application/octet-stream]
Saving to: ‘data/script.ipynb.1264’

script.ipynb.1264   100%[===================>]  18.09K  --.-KB/s    in 0.05s   

2018-04-24 13:55:13 (359 KB/s) - ‘data/script.ipynb.1264’ saved [18524/18524]



100%|█████████▉| 1844/1845 [17:31<00:00,  1.75it/s]

--2018-04-24 13:55:14--  https://www.kaggle.com/kernels/scriptcontent/2124858/download
Resolving www.kaggle.com (www.kaggle.com)... 168.62.224.124
Connecting to www.kaggle.com (www.kaggle.com)|168.62.224.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24929 (24K) [application/octet-stream]
Saving to: ‘data/script.ipynb.1265’

script.ipynb.1265   100%[===================>]  24.34K  --.-KB/s    in 0.05s   

2018-04-24 13:55:14 (474 KB/s) - ‘data/script.ipynb.1265’ saved [24929/24929]



100%|██████████| 1845/1845 [17:31<00:00,  1.75it/s]


## 4. Clean non-Python files from downloaded

In [89]:
for filename in os.listdir("data"):
    if filename == ".DS_Store":
        pass
    
    name_parts = filename.split(".")
    if len(name_parts) == 2:
        name_parts.append("0")
        
    if name_parts[1] in ['ipynb', 'py']:
        new_name = name_parts[0] + name_parts[2] + "." + name_parts[1]
        new_name = 'r' + new_name if name_parts[1] == 'py' else new_name
        os.rename('data/' + filename, 'data/' + new_name)

    else:
        os.remove('data/' + filename)

## Tokenize Notebooks and Scripts

In [67]:
# Analyze python script
pd_search_tokens = ["." + p[0] for p in inspect.getmembers(pd) if p[1].__class__.__name__ == 'function'] + ['.DataFrame']
df_search_tokens = ["." + p[0] for p in inspect.getmembers(pd.DataFrame) if p[1].__class__.__name__ == 'function']
search_tokens = pd_search_tokens + df_search_tokens

def parse_script(script_name):
    with open(script_name) as script_file:
        code = script_file.read().splitlines()

    ps = {}
    for line in code:
        for token in search_tokens:
            for m in re.finditer(token, line):
                found = line[m.start() - 2: m.end()]
                if '.' in found and (found[:2] == 'pd' or found[:2] == 'df'):
                    if not (len(line) > m.end() and (line[m.end()].isalnum() or line[m.end()] == '_')):
                        if found not in ps:
                            ps[found] = 0
                        ps[found] += 1
    
    return ps

In [68]:
parse_script('data/script1092.py')

{'pd.read_csv': 3, 'pd.to_datetime': 2}

In [69]:
!rm data/.DS_Store

all_methods = []
for filename in tqdm(os.listdir("data")):
    if filename.split(".")[1] == 'py':
        all_methods.append(parse_script('data/' + filename))
    else:
        pfilename = 'data/' + filename
        cfilename = filename.split(".")[0] + '.py'
        if cfilename not in os.listdir("data"):
            !jupyter nbconvert --to=python $pfilename
            time.sleep(1)
        try:
            all_methods.append(parse_script('data/' + cfilename))
        except:
            pass

rm: data/.DS_Store: No such file or directory


100%|██████████| 3028/3028 [06:59<00:00,  7.23it/s]


In [89]:
all_methods = [a for a in all_methods if a]
len(all_methods)

1562

In [90]:
import json
methods_write = open('results/methods.txt', 'w')
methods_write.write(json.dumps(all_methods))

94020

## Calculate total method usage

In [91]:
methods_read = []
with open('results/methods.txt', 'r') as method_file_read:
    methods_read = json.loads(method_file_read.read())

methods_read[:5]

[{'df.fillna': 1, 'pd.merge': 20, 'pd.read_csv': 7, 'pd.to_datetime': 1},
 {'pd.DataFrame': 2, 'pd.concat': 1, 'pd.get_dummies': 1, 'pd.read_table': 2},
 {'pd.read_csv': 1},
 {'pd.DataFrame': 2,
  'pd.concat': 1,
  'pd.get_dummies': 1,
  'pd.read_pickle': 1,
  'pd.read_table': 4,
  'pd.to_pickle': 2},
 {'pd.DataFrame': 1, 'pd.read_csv': 5}]

In [92]:
base = {}
for dict_method in all_methods:
    for k in dict_method:
        if k not in base:
            base[k] = 0 
        base[k] += dict_method[k]

In [93]:
m, c = [], []
for method, count in base.items():
    m.append(method)
    c.append(count)
    
mFreq = pd.DataFrame(data = {'Method' : m, 'Count': c}).set_index('Method').sort_values('Count', ascending=False)
mFreq.to_csv('results/results.csv')
mFreq.head(10)

,Count
Method,
pd.read_csv,3887
pd.DataFrame,2491
pd.merge,1672
pd.concat,1061
df.groupby,870
pd.to_datetime,522
df.head,431
df.drop,427
pd.get_dummies,425


In [94]:
impl = pd.read_csv('results/annotResults.csv')
impl.head()

,Method,Count,Finished
0,pd.read_csv,1422,1.0
1,pd.DataFrame,886,1.0
2,df.append,792,1.0
3,df.mean,783,1.0
4,df.head,783,1.0


## Calculate notebook coverage

In [95]:
cleaned = ['pd.groupby', 'pd.merge', 'df.mod', 'df.ge', 'pd.isnull']

def scriptCovered(script):
    scriptMethods = [l for l in list(script.keys()) if l not in cleaned]
    v = []
    failed = []
    for method in scriptMethods:
        res = impl[impl['Method'] == method]['Finished']
        if len(res) > 0:
            v.append(res.values[0])
        if len(res) == 0 or res.values[0] == 0.0:
            failed.append(method)
    
    return v.count(1.0) == len(v), failed


def processCoverage(coverageResults):
    """
    Return count of covered and uncovered scripts
            and frequency of uncovered methods
    """
    status, uncoveredMethods = [c[0] for c in coverageResults], [c[1] for c in coverageResults]
    numCovered = status.count(True)
    perCovered = numCovered/len(coverageResults)
    
    uncoveredFreq = {}
    for ums in uncoveredMethods:
        for um in ums:
            if um not in uncoveredFreq:
                uncoveredFreq[um] = 0
            uncoveredFreq[um] += 1
            
    return numCovered, perCovered, uncoveredFreq

In [96]:
coverageResults = [scriptCovered(m) for m in tqdm(all_methods)]

100%|██████████| 1562/1562 [00:03<00:00, 519.05it/s]


In [97]:
missing = (pd.DataFrame
           .from_dict(data=processCoverage(coverageResults)[2], orient='index')
           .rename(columns={0:'count'})
           .sort_values('count', ascending=False))

missing

,count
pd.notnull,14
df.pivot_table,3
pd.sort_values,1
pd.head,1
pd.isna,1
df.unique,1
pd.append,1
pd.describe,1


In [98]:
processCoverage(coverageResults)

(1475,
 0.9443021766965429,
 {'df.pivot_table': 3,
  'df.unique': 1,
  'pd.append': 1,
  'pd.describe': 1,
  'pd.head': 1,
  'pd.isna': 1,
  'pd.notnull': 14,
  'pd.sort_values': 1})